<a href="https://colab.research.google.com/github/kouseizack/NLP/blob/master/py_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
cd /content/drive/My Drive/workpy

/content/drive/My Drive/workpy


In [32]:
!pip install PyPDF4
!pip install tabula-py
!pip install pdf2image
!apt-get install poppler-utils
!pip install pandas

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [33]:
#Remove watermark from the image
def removeWatermark(wm_text, inputFile, outputFile):
    from PyPDF4 import PdfFileReader, PdfFileWriter
    from PyPDF4.pdf import ContentStream
    from PyPDF4.generic import TextStringObject, NameObject
    from PyPDF4.utils import b_

    with open(inputFile, "rb") as f:
        source = PdfFileReader(f, "rb")
        output = PdfFileWriter()

        for page in range(source.getNumPages()):
            page = source.getPage(page)
            content_object = page["/Contents"].getObject()
            content = ContentStream(content_object, source)

            for operands, operator in content.operations:
                if operator == b_("Tj"):
                    text = operands[0]

                    if isinstance(text, str) and text.startswith(wm_text):
                        operands[0] = TextStringObject('')

            page.__setitem__(NameObject('/Contents'), content)
            output.addPage(page)

        with open(outputFile, "wb") as outputStream:
            output.write(outputStream)

wm_text = 'EBanking'
inputFile = r'canopy_technical_test_input.pdf'
outputFile = r"output.pdf"
removeWatermark(wm_text, inputFile, outputFile)

In [34]:
'''Use tabula to read table content.
    Process data frame to be converted to excrel later'''
from tabula import read_pdf
import pandas as pd
df = read_pdf("output.pdf")

df[0].loc[0, "Booking Details"]
labels = {df[0].iloc[0 , i]:[] for i in range(len(df[0].columns))}
print (labels)

for j in range(1 , len(df[0])):
  for k in range(len(df[0].columns)):
    if(k == 0):
      if(not pd.isnull(df[0].iloc[j , k])):
        labels["Booking Date"].append(df[0].iloc[j , k])
    if(k == 1):
      if(not pd.isnull(df[0].iloc[j , k])):
        labels["Txn Date"].append(df[0].iloc[j , k])
    if(k == 2):
      if(pd.isnull(df[0].iloc[j , 0])):
        labels["Booking Text"][-1] += df[0].iloc[j , k]
      else:
        labels["Booking Text"].append(df[0].iloc[j , k])
    if(k == 3):
      if(not pd.isnull(df[0].iloc[j , 0])):
        if(pd.isnull(df[0].iloc[j , k])):
          labels["Value Date"].append("")
        else:
          labels["Value Date"].append(df[0].iloc[j , k])
    if(k == 4):
      if(not pd.isnull(df[0].iloc[j , 0])):
        if(pd.isnull(df[0].iloc[j , k])):
          labels["Debit"].append("")
        else:
          labels["Debit"].append(df[0].iloc[j , k])
    if(k == 5):
      if(not pd.isnull(df[0].iloc[j , 0])):
        if(pd.isnull(df[0].iloc[j , k])):
          labels["Credit"].append("")
        else:
          labels["Credit"].append(df[0].iloc[j , k])
    if(k == 6):
      if(not pd.isnull(df[0].iloc[j , 0])):
        if(pd.isnull(df[0].iloc[j , k])):
          labels["Balance"].append("")
        else:
          labels["Balance"].append(df[0].iloc[j , k])
  
print (labels)

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Aug 14, 2020 12:19:47 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Aug 14, 2020 12:19:47 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



{'Booking Date': [], 'Txn Date': [], 'Booking Text': [], 'Value Date': [], 'Debit': [], 'Credit': [], 'Balance': []}
{'Booking Date': ['31.03.2018', '01.04.2018', '06.04.2018', '06.04.2018', '06.04.2018', '06.04.2018', '09.04.2018', '09.04.2018', '10.04.2018', '10.04.2018'], 'Txn Date': ['31.03.2018', '01.04.2018', '06.04.2018', '06.04.2018', '06.04.2018', '06.04.2018', '09.04.2018', '09.04.2018', '10.04.2018', '10.04.2018'], 'Booking Text': ['Initial Balance', 'VALUE DATED BALANCE BROUGHT FORWARD', 'INTEREST-FIXED TERM LOANContract No: 3001-AA18091ZN72C|Interest rate:0.810000%|Capital: 1,000,000,000.00|Period: 16.03.2018 -06.04.2018|Days: 21/360', 'INTEREST-FIXED TERM LOANContract No: 3001-AA1809166QNF|Interest rate:0.810000%|Capital: 1,000,000,000.00|Period: 23.03.2018 -06.04.2018|Days: 14/360', 'FOREX SPOTEUR/JPY 130.7271', 'FOREX SPOTEUR/JPY 130.7021', 'INTEREST-FIXED TERM LOANContract No: 3001-AA18091NTT8N|Interest rate:0.810000%|Capital: 500,000,000.00|Period: 26.03.2018 -09.04.2

In [36]:
#Convert dataframe to excel and save in output.xlsx
import pandas as pd
df = pd.DataFrame(labels)
df.to_excel("output.xlsx") 
df

,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
0,31.03.2018,31.03.2018,Initial Balance,,,,0.00
1,01.04.2018,01.04.2018,VALUE DATED BALANCE BROUGHT FORWARD,01.04.2018,,"180,431,640.00","180,431,640.00"
2,06.04.2018,06.04.2018,INTEREST-FIXED TERM LOANContract No: 3001-AA18...,06.04.2018,"472,500.00",,"179,959,140.00"
3,06.04.2018,06.04.2018,INTEREST-FIXED TERM LOANContract No: 3001-AA18...,06.04.2018,"315,000.00",,"179,644,140.00"
4,06.04.2018,06.04.2018,FOREX SPOTEUR/JPY 130.7271,06.04.2018,,"472,500.00","180,116,640.00"
5,06.04.2018,06.04.2018,FOREX SPOTEUR/JPY 130.7021,06.04.2018,,"315,000.00","180,431,640.00"
6,09.04.2018,09.04.2018,INTEREST-FIXED TERM LOANContract No: 3001-AA18...,09.04.2018,"157,500.00",,"180,274,140.00"
7,09.04.2018,09.04.2018,FOREX SPOTEUR/JPY 131.1407,09.04.2018,,"157,500.00","180,431,640.00"
8,10.04.2018,10.04.2018,INTEREST-FIXED TERM LOANContract No: 3001-AA18...,10.04.2018,"157,500.00",,"180,274,140.00"
9,10.04.2018,10.04.2018,FOREX SPOT,10.04.2018,,"157,500.00","180,431,640.00"
